In [1]:
import numpy as np
import random
from scipy.stats import bernoulli
import scipy
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from tqdm.notebook import tqdm

In [2]:
class Environment(object):
    '''General RL environment'''

    def __init__(self):
        pass

    def reset(self):
        pass

    def advance(self, action):
        '''
        Moves one step in the environment.
        Args:
            action
        Returns:
            reward - double - reward
            newState - int - new state
            pContinue - 0/1 - flag for end of the episode
        '''
        return 0, 0, 0

class Opt_Policy(object):
    '''
    For Computing Q*, the optimal q_values. This code was written with RiverSwim in mind.
    '''
    def __init__(self,env):
        self.env = env
        self.Q = np.zeros((self.env.epLen+1,self.env.nState,self.env.nAction))
        self.V = np.zeros((self.env.epLen+1,self.env.nState))
    
    def update(self):
        Q = np.zeros((self.env.epLen + 1,self.env.nState,self.env.nAction))
        V = np.zeros((self.env.epLen + 1,self.env.nState))
        for h in range(self.env.epLen-1,-1,-1):
            for s in range(self.env.nState):
                for a in range(self.env.nAction):
                    reward = env.R[s,a][0]
                    '''
                    for s_ in range(self.env.nState):
                        #print(s,a,s_)
                        reward += self.env.R[s,a,s_][0]*self.env.P[s,a][s_]
                    '''
                    p = env.P[s,a]
                    Q[h,s,a] = reward + np.inner(p,V[h+1,:])
                V[h,s] = max(Q[h,s,:])
        self.Q = Q.copy()
        self.V = V.copy()
    
    def act(self,s,h):
        return env.argmax(self.Q[h,s,:])
    

In [3]:
def make_riverSwim(epLen=20, nState=5):
    '''
    Makes the benchmark RiverSwim MDP.
    Args:
        NULL - works for default implementation
    Returns:
        riverSwim - Tabular MDP environment '''
    nAction = 2
    R_true = {}
    P_true = {}
    states = {}
    for s in range(nState):
        states[(s)] = 0.0
        for a in range(nAction):
            R_true[s, a] = (0, 0)
            P_true[s, a] = np.zeros(nState)

    # Rewards
    R_true[0, 0] = (5/1000, 0)
    R_true[nState - 1, 1] = (1, 0)

    # Transitions
    for s in range(nState):
        P_true[s, 0][max(0, s-1)] = 1.

    for s in range(1, nState - 1):
        P_true[s, 1][min(nState - 1, s + 1)] = 0.3
        P_true[s, 1][s] = 0.6
        P_true[s, 1][max(0, s-1)] = 0.1

    P_true[0, 1][0] = 0.3
    P_true[0, 1][1] = 0.7
    P_true[nState - 1, 1][nState - 1] = 0.9
    P_true[nState - 1, 1][nState - 2] = 0.1

    riverSwim = TabularMDP(nState, nAction, epLen)
    riverSwim.R = R_true
    riverSwim.P = P_true
    riverSwim.states = states
    riverSwim.reset()

    return riverSwim

def make_MDP(epLen=2,nBottom = 24):
    '''
    Makes the benchmark RiverSwim MDP.
    Args:
        NULL - works for default implementation
    Returns:
        riverSwim - Tabular MDP environment '''
    nState = nBottom + 3
    nAction = 2
    R_true = {}
    P_true = {}
    states = {}
    for s in range(nState):
        states[(s)] = 0.0
        for a in range(nAction):
            R_true[s, a] = (0, 0)
            P_true[s, a] = np.zeros(nState)

    # Rewards
    R_true[1,0] = (0, 0)
    R_true[1,1] = (0, 0)
    R_true[2,0] = (1, 0)
    R_true[2,1] = (1, 0)
    n = int(nBottom/4)
    #Transitions 
    P_true[0,0][1] = 1.0
    P_true[0,1][2] = 1.0
    P_true[1,0][3:3+n] = 1/n
    P_true[1,1][3+n:3+2*n] = 1/n
    P_true[2,0][3+2*n:3+3*n] = 1/n
    P_true[2,1][3+3*n:3+4*n] = 1/n
    

    MDP = TabularMDP(nState, nAction, epLen)
    MDP.R = R_true
    MDP.P = P_true
    MDP.states = states
    MDP.reset()

    return MDP

class TabularMDP(Environment):
    '''
    Tabular MDP
    R - dict by (s,a) - each R[s,a] = (meanReward, sdReward)
    P - dict by (s,a) - each P[s,a] = transition vector size S
    '''

    def __init__(self, nState, nAction, epLen):
        '''
        Initialize a tabular episodic MDP
        Args:
            nState  - int - number of states
            nAction - int - number of actions
            epLen   - int - episode length
        Returns:
            Environment object
        '''

        self.nState = nState
        self.nAction = nAction
        self.epLen = epLen

        self.timestep = 0
        self.state = 0

        # Now initialize R and P
        self.R = {}
        self.P = {}
        self.states = {}
        for state in range(nState):
            for action in range(nAction):
                self.R[state, action] = (1, 1)
                self.P[state, action] = np.ones(nState) / nState
                
    def reset(self):
        "Resets the Environment"
        self.timestep = 0
        self.state = 0
        
    def advance(self,action):
        '''
        Move one step in the environment
        Args:
        action - int - chosen action
        Returns:
        reward - double - reward
        newState - int - new state
        episodeEnd - 0/1 - flag for end of the episode
        '''
        if self.R[self.state, action][1] < 1e-9:
            # Hack for no noise
            reward = self.R[self.state, action][0]
        else:
            reward = np.random.normal(loc=self.R[self.state, action][0],
                                      scale=self.R[self.state, action][1])
        #print(self.state, action, self.P[self.state, action])
        newState = np.random.choice(self.nState, p=self.P[self.state, action])
        
        # Update the environment
        self.state = newState
        self.timestep += 1

        episodeEnd = 0
        if self.timestep == self.epLen:
            episodeEnd = 1
            #newState = None
            self.reset()

        return reward, newState, episodeEnd
    
    def argmax(self,b):
        #print(b)
        return np.random.choice(np.where(b == b.max())[0])

In [4]:
class UCRL_VTR(object):
    '''
    Algorithm 1 as described in the paper Model-Based RL with
    Value-Target Regression
    The algorithm assumes that the rewards are in the [0,1] interval.
    '''
    def __init__(self,env,K):
        self.env = env
        self.K = K
        # A unit test that randomly explores for a period of time then learns from that experience
        # Here self.random_explore is a way to select a period of random exploration.
        # When the current episode k > total number of episodes K divided by self.random_explore
        # the algorithm switches to the greedy action with respect to its action value Q(s,a).
        # Here the dimension (self.d) for the Tabular setting is |S x A x S| as stated in Appendix B
        self.d = self.env.nState * self.env.nAction * self.env.nState
        # In the tabular setting the basis models is just the dxd identity matrix, see Appendix B
        self.P_basis = np.identity(self.d)
        #Our Q-values are initialized as a 2d numpy array, will eventually convert to a dictionary
        self.Q = {(h,s,a): 0.0 for h in range(self.env.epLen) for s in self.env.states.keys() \
                   for a in range(self.env.nAction)}
        #Our State Value function is initialized as a 1d numpy error, will eventually convert to a dictionary
        self.V = {(h,s): 0.0 for s in self.env.states.keys() for h in range(env.epLen + 1)} # self.V[env.epLen] stays zero
        #self.create_value_functions()
        #The index of each (s,a,s') tuple, see Appendix B
        self.sigma = {}
        self.state_idx = {}
        self.createIdx()
        #See Step 2, of algorithm 1
#         self.M = env.epLen**2*self.d*np.identity(self.d)
        # For use in the confidence bound bonus term, see Beta function down below
        self.lam = 1.0
        #Self.L is no longer need, but will keep for now.
        self.L = 1.0
        self.M = np.identity(self.d)*self.lam
        self.Minv = np.identity(self.d)*(1/self.lam)
        #See Step 2
        self.w = np.zeros(self.d)
        #See Step 2
        self.theta = np.dot(self.Minv,self.w)
        #See Step 3
        self.delta = 1/self.K
        #m_2 >= the 2-norm of theta_star, see Bandit Algorithms Theorem 20.5
        #self.error()
        #self.m_2 = np.linalg.norm(self.true_p) + 0.1
        self.m_2 = np.sqrt(self.env.nState*self.env.nAction)
        self.d1 = env.nState * env.nAction




    def feature_vector(self,s,a,h):
        '''
        Returning sum_{s'} V[h+1][s'] P_dot(s'|s,a),
        with V stored in self.
        Inputs:
            s - the state
            a - the action
            h - the current timestep within the episode
        '''
        sums = np.zeros(self.d)
        for s_ in self.env.states.keys():
            #print(s,s_)
            sums += self.V[h+1,s_] * self.P_basis[self.sigma[(s,a,s_)]]
        return sums

    def proj(self, x, lo, hi):
        '''Projects the value of x into the [lo,hi] interval'''
        return max(min(x,hi),lo)

    def update_Q(self,s,a,k,h):
        '''
        A function that updates both Q and V, Q is updated according to equation 4 and
        V is updated according to equation 2
        Inputs:
            s - the state
            a - the action
            k - the current episode
            h - the current timestep within the episode
        Currently, does not properly compute the Q-values but it does seem to learn theta_star
        '''
        #Here env.R[(s,a)][0] is the true reward from the environment
        # Alex's code: X = self.X[h,:]
        # Suggested code:
        X = self.feature_vector(s,a,h)
        self.Q[h,s,a] = self.proj(self.env.R[(s,a)][0] + np.dot(X,self.theta) + self.Beta(h) \
            * np.sqrt(np.dot(np.dot(np.transpose(X),self.Minv),X)), 0, self.env.epLen)
        self.V[h,s] = max(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))

    def update_Qend(self,k):
        '''
        A function that updates both Q and V at the end of each episode, see step 16 of algorithm 1
        Inputs:
            k - the current episode
        '''
        #step 16
        for h in range(self.env.epLen-1,-1,-1):
            for s in self.env.states.keys():
                for a in range(self.env.nAction):
                    #Here env.R[(s,a)][0] is the true reward from the environment
                    # Alex's code: X = self.X[h,:]
                    # Suggested code:
                    self.update_Q(s,a,k,h)
                self.V[h,s] = max(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))

    def update_stat(self,s,a,s_,h):
        '''
        A function that performs steps 9-13 of algorithm 1
        Inputs:
            s - the current state
            a - the action
            s_ - the next state
            k - the current episode
            h - the timestep within episode when s was visited (starting at zero)
        '''
        #Step 10
#         self.X[h,:] = self.feature_vector(s,a,h) # do not need to store this
        X = self.feature_vector(s,a,h)
        #Step 11
        y = self.V[h+1,s_]
#         if s_ != None:
#             y = self.V[h+1][s_]
#         else:
#             y = 0.0
        #Step 12
        self.M = self.M + np.outer(X,X)
        self.Minv = self.Minv - np.dot((np.outer(np.dot(self.Minv,X),X)),self.Minv) / \
                    (1 + np.dot(np.dot(X,self.Minv),X))
        #Step 13
        self.w = self.w + y*X

    def update_param(self):
        '''
        Updates our approximation of theta_star at the end of each episode, see
        Step 15 of algorithm1
        '''
        #Step 15
        self.theta = np.matmul(self.Minv,self.w)

    def act(self,s,h,k):
        '''
        Returns the greedy action with respect to Q_{h,k}(s,a) for a \in A
        see step 8 of algorithm 1
        Inputs:
            s - the current state
            h - the current timestep within the episode
        '''
        
        return self.env.argmax(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))
        

    def createIdx(self):
        '''
        A simple function that creates sigma according to Appendix B.
        Here sigma is a dictionary who inputs is a tuple (s,a,s') and stores
        the interger index to be used in our basis model P.
        '''
        i = 0
        j = 0
        k = 0
        for s in self.env.states.keys():
            self.state_idx[s] = int(j)
            j += 1
            for a in range(self.env.nAction):
                for s_ in self.env.states.keys():
                    self.sigma[(s,a,s_)] = int(i)
                    i += 1

    def Beta(self,h):
        '''
        A function that return Beta_k according to Algorithm 1, step 3
        '''
        #Step 3
        #Confidence bound from Appendix F/Chapter 20 of Bandit Algorithms Chpt 20 (Lattimore/Szepesvari).
        first = np.sqrt(self.lam)*self.m_2
        (sign, logdet) = np.linalg.slogdet(self.M)
        det = sign * logdet
        second = (self.env.epLen-h)/2*np.sqrt(2*np.log(1/self.delta) + min(det,pow(10,10)) - np.log(pow(self.lam,self.d)))
        return first + second
    
    def getweightedL1(self):
        self.weights = self.count
        self.true_p = np.zeros((self.env.nState,self.env.nAction,self.env.nState))
        for s in range(self.env.nState):
            for a in range(self.env.nAction):
                for s_ in range(self.env.nState):
                    self.true_p[s,a,s_] = self.env.P[s,a][s_]
                    
                    #for numerical stability
                    if sum(self.count[s,a,:]) == 0:
                        self.weights[s,a,s_] = self.count[s,a,s_]/1.0
                        
                    else:
                        self.weights[s,a,s_] = self.count[s,a,s_]/sum(self.count[s,a,:])
                        
        self.weights = self.weights.reshape(env.nState*env.nAction*env.nState)
        self.true_p = self.true_p.reshape(env.nState*env.nAction*env.nState)
        temp = 0
        for i in range(env.nState*env.nAction*env.nState):
            temp += abs(self.theta[i]-self.true_p[i])*self.weights[i]
        return temp

    def run(self):
        '''
        Simulates the agent interacting with the environment over K episodes
        Input: Nothing
        Output: A Kx1 reward vector and a Kx1 model error vector
        '''
        R = []
        reward = 0.0
        #Stores counts for use in weighted L1 norm
        self.count = np.zeros((env.nState,env.nAction,env.nState))
        self.model_error = np.zeros(self.K)
        print(self.name())
        for k in tqdm(range(1,self.K+1)):
            self.env.reset()
            done = 0
            while done != 1:
                s = self.env.state
                h = self.env.timestep
                a = self.act(s,h,k)
                r,s_,done = self.env.advance(a)
                self.count[s,a,s_] += 1
                reward += r
                self.update_stat(s,a,s_,h)
            self.update_param()
            self.update_Qend(k)
            R.append(reward)
            self.model_error[k-1] = self.getweightedL1()
        return R

    def name(self):
        return 'Running: UCRL_VTR'


In [5]:
class EG_MatrixRL(object):
    def __init__(self,env,N,eps):
        self.env = env
        #See Step 3 of Algorithm 1
        self.N = N
        self.eps = eps
        #The dimensionality of phi(s,a), see Step 2 or Assumption 1
        self.d1 = self.env.nState * self.env.nAction
        #The dimensionality of psi(s'), see Step 2 or Assumption 1
        self.d2 = self.env.nState
        #Step 2
        self.features_state_action = {(s,a): np.zeros(self.d1) for s in self.env.states.keys() for a in range(self.env.nAction)}
        #Step 3
        self.features_next_state = {(s): np.zeros(self.d2) for s in self.env.states.keys()}
        # A hack for using numpy's linear algebra functions in Step 4
        self.features_next_state_mat = np.identity(self.d2)
        # Creates the Identity Matrix for a dictionary
        self.createIdentity()
        # Initialize our Q matrix
        self.Q = {(h,s,a): 0.0 for h in range(self.env.epLen) for s in self.env.states.keys() \
                   for a in range(self.env.nAction)}
        #Step 4, this step maybe unnecessary since we are using Sherman-Morrison
        self.A = np.identity(self.d1)
        #For use in the Sherman-Morrison Update
        self.Ainv = np.linalg.inv(self.A)
        #Step 4
        self.M = np.zeros((self.d1,self.d2))
        self.M_mat = np.zeros((self.N+1,self.d1*self.d2))
        self.M_mat[0,:] = self.M.reshape(self.d1*self.d2)
        #See Assumptions 2,2' and Theorem 1, this equals 1 in the tabular case
        self.C_phi = 1.0
        # See Assumption 2'(Stronger Feature Regularity), and consider the case when v_1 = v_2 = ....
        self.C_psi = np.sqrt(env.nState)
        # See Theorem 1
        self.C_M = 1.0
        # See Theorem 1
        self.C_psi_ = 1.0
        # This value scales our confidence interval, must be > 0
        self.c = 1.0
        # For use in updating M_n, see Step 13 and Eqn (2)
        self.sums = np.zeros((self.d1,self.d2))
        #Creates K_psi, see Section 3.1: Estimating the core matrix.
        self.createK()
        self.lam = 1.0
        self.delta = 1/self.N



    def createK(self):
        '''
        A function that creates K_psi and (K_psi)^-1 in computing M_n
        '''
        self.K = np.zeros((self.d2,self.d2))
        for s_ in self.env.states.keys():
            self.K = self.K + np.outer(self.features_next_state[s_],self.features_next_state[s_])
        self.Kinv = np.linalg.inv(self.K)

    def act(self,s,h):
        '''
        Returns the eps-greedy action with respect to Q_{h,k}(s,a) for a \in A
        Inputs:
            s - the current state
            h - the current timestep within the episode
        '''
        
        c = np.random.uniform()
        if c >= self.eps:
            return self.env.argmax(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))
        else:
            return np.random.choice([0,1])
        

    def createIdentity(self):
        '''
        A function that creates the Identity Matrix for a Dictionary
        '''
        i = 0
        for key in self.features_state_action.keys():
            self.features_state_action[key][i] = 1
            i += 1
        j = 0
        for key in self.features_next_state.keys():
            self.features_next_state[key][j] = 1
            j += 1

    def proj(self, x, lo, hi):
        '''Projects the value of x into the [lo,hi] interval'''
        return max(min(x,hi),lo)

    def compute_Q(self,n):
        '''
        A function that computes the Optimisic Q-Values.
        '''
        Q = {(h,s,a): 0.0 for h in range(self.env.epLen) for s in self.env.states.keys() \
                   for a in range(self.env.nAction)}
        V = {h: np.zeros(self.env.nState) for h in range(self.env.epLen + 1)}
        for h in range(self.env.epLen-1,-1,-1):
            for s in self.env.states.keys():
                for a in range(self.env.nAction):
                    #For use in computing Q, like in UCRL_VTR we have access to the true reward function.
                    r = self.env.R[s,a][0]

                    value = np.dot(np.matmul(np.dot(self.features_state_action[s,a].T,self.M),\
                            self.features_next_state_mat),V[h+1])
                    Q[h,s,a] = self.proj(r+value,0,self.env.epLen)
                V[h][s] = (1-self.eps/2)*max(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)])) \
                        + (self.eps/2)*min(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))
        self.Q = Q.copy()

        

    def update_core_matrix(self,s,a,s_):
        
        #Sherman Morrison Update (Rich's book Eqn 9.22)
        self.Ainv = self.Ainv - np.dot((np.outer(np.dot(self.Ainv,self.features_state_action[s,a]) \
                 ,self.features_state_action[s,a])),self.Ainv) / \
                    (1 + np.dot(np.dot(self.features_state_action[s,a],self.Ainv),self.features_state_action[s,a]))
        
        self.sums = self.sums + np.outer(self.features_state_action[s,a],self.features_next_state[s_])
        #Here (K_psi)^-1 was pre-computed during the initialization. 
        self.M = np.matmul(np.matmul(self.Ainv,self.sums),self.Kinv)
        

    
    def name(self):
        return 'Running: EG_MatrixRL'
    
    def getweightedL1(self):
        '''
        Returns the weighed L1 norm of the current estimated model and the true environment model
        '''
        self.weights = self.count
        self.true_p = np.zeros((self.env.nState,self.env.nAction,self.env.nState))
        for s in range(self.env.nState):
            for a in range(self.env.nAction):
                for s_ in range(self.env.nState):
                    self.true_p[s,a,s_] = self.env.P[s,a][s_]
                    
                    #for numerical stability
                    if sum(self.count[s,a,:]) == 0:
                        self.weights[s,a,s_] = self.count[s,a,s_]/1.0
                        
                    else:
                        self.weights[s,a,s_] = self.count[s,a,s_]/sum(self.count[s,a,:])
                        
        self.weights = self.weights.reshape(env.nState*env.nAction*env.nState)
        self.true_p = self.true_p.reshape(env.nState*env.nAction*env.nState)
        self.theta = self.M.reshape(env.nState*env.nAction*env.nState)
        temp = 0
        for i in range(env.nState*env.nAction*env.nState):
            temp += abs(self.theta[i]-self.true_p[i])*self.weights[i]
        return temp

    def run(self):
        '''
        Simulates the agent interacting with the environment over K episodes
        Input: Nothing
        Output: A Kx1 reward vector and a Kx1 model error vector
        '''
        R = 0
        Rvec = []
        Reg = []
        #Stores count to be used in Weighted L1 Norm
        self.count = np.zeros((env.nState,env.nAction,env.nState))
        self.model_error = np.zeros(self.N)
        #print(self.name())
        for n in (range(1,self.N+1)):
            self.env.reset()
            done = 0
            self.compute_Q(n)
            while not done:
                s = self.env.state
                h = self.env.timestep
                a = self.act(s,h)
                r,s_,done = self.env.advance(a)
                self.count[s,a,s_] += 1
                R += r
                self.update_core_matrix(s,a,s_)
            Rvec.append(R)
            self.model_error[n-1] = self.getweightedL1()
        return Rvec

In [6]:
class UC_MatrixRL(object):
    def __init__(self,env,N):
        self.env = env
        #See Step 3 of Algorithm 1
        self.N = N
        #The dimensionality of phi(s,a), see Step 2 or Assumption 1
        self.d1 = self.env.nState * self.env.nAction
        #The dimensionality of psi(s'), see Step 2 or Assumption 1
        self.d2 = self.env.nState
        #Step 2
        self.features_state_action = {(s,a): np.zeros(self.d1) for s in self.env.states.keys() for a in range(self.env.nAction)}
        #Step 3
        self.features_next_state = {(s): np.zeros(self.d2) for s in self.env.states.keys()}
        # A hack for using numpy's linear algebra functions in Step 4
        self.features_next_state_mat = np.identity(self.d2)
        # Creates the Identity Matrix for a dictionary
        self.createIdentity()
        # Initialize our Q matrix
        self.Q = {(h,s,a): 0.0 for h in range(self.env.epLen) for s in self.env.states.keys() \
                   for a in range(self.env.nAction)}
        #Step 4, this step may be unnecessary since we are using Sherman-Morrison
        self.A = np.identity(self.d1)
        #For use in the Sherman-Morrison Update
        self.Ainv = np.linalg.inv(self.A)
        #Step 4
        self.M = np.zeros((self.d1,self.d2))
        #See Assumptions 2,2' and Theorem 1, this equals 1 in the tabular case
        self.C_phi = 1.0
        # See Assumption 2'(Stronger Feature Regularity), and consider the case when v_1 = v_2 = ....
        self.C_psi = 1.0
        # See Theorem 1
        self.C_M = 1.0
        # See Theorem 1
        self.C_psi_ = 1.0
        # This value scales our confidence interval, must be > 0
        self.c = 1.0
        # For use in updating M_n, see Step 13 and Eqn (2)
        self.sums = np.zeros((self.d1,self.d2))
        #Creates K_psi.
        self.createK()
        self.lam = 1.0
        self.delta = 1/self.N

    def createK(self):
        '''
        A function that creates K_psi and (K_psi)^-1 in computing M_n
        '''
        
        self.K = np.zeros((self.d2,self.d2))
        for s_ in self.env.states.keys():
            self.K = self.K + np.outer(self.features_next_state[s_],self.features_next_state[s_])
        self.Kinv = np.linalg.inv(self.K)

    def act(self,s,h):
        '''
        A function that returns the argmax of Q given the state and timestep
        '''
        return self.env.argmax(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))

    def createIdentity(self):
        '''
            A function that creates the Identity Matrix for a Dictionary
        '''
        i = 0
        for key in self.features_state_action.keys():
            self.features_state_action[key][i] = 1
            i += 1
        j = 0
        for key in self.features_next_state.keys():
            self.features_next_state[key][j] = 1
            j += 1

    def proj(self, x, lo, hi):
        '''Projects the value of x into the [lo,hi] interval'''
        return max(min(x,hi),lo)

    def compute_Q(self,n):
        '''
        A function that computes the Optimisic Q-Values, see step 6 and Equations 4,8.
        '''
        Q = {(h,s,a): 0.0 for h in range(self.env.epLen) for s in self.env.states.keys() \
                   for a in range(self.env.nAction)}
        V = {h: np.zeros(self.env.nState) for h in range(self.env.epLen + 1)}
        for h in range(self.env.epLen-1,-1,-1):
            for s in self.env.states.keys():
                for a in range(self.env.nAction):
                    #For use in computing Q, like in UCRL_VTR we have access to the true reward function.
                    r = self.env.R[s,a][0]

                    value = np.dot(np.matmul(np.dot(self.features_state_action[s,a].T,self.M),\
                            self.features_next_state_mat),V[h+1])
                    
                    bonus = 2 * self.C_psi * self.env.epLen * self.Beta(h) * np.sqrt(np.dot(\
                           np.dot(self.features_state_action[s,a],self.Ainv),self.features_state_action[s,a]))
                    # Computing the optimistic Q-values as according to Eqn (8).
                    Q[h,s,a] = self.proj(r+value+bonus,0,self.env.epLen)
                V[h][s] = max(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))
        self.Q = Q.copy()


    def Beta(self,h):
        '''
        A function that return Beta_k according to Theorem 20.5 in Bandits Algorithms (Lattimore/Szepesvari)/
        Appendix F of the VTR paper.
        Log Determinate was used for numerical stability purposes
        '''
        #Confidence bound from Appendix F of the our VTR-paper.
        first = np.sqrt(self.lam)*np.sqrt(self.C_M*self.d1)
        (sign, logdet) = np.linalg.slogdet(scipy.linalg.sqrtm(self.A))
        det = sign * logdet
        second = (self.env.epLen - h)/2*np.sqrt(2*np.log(1/self.delta) + min(det,pow(10,10)) - np.log(pow(self.lam,self.d1)))
        #print(det)
        #print(first+second)
        return first + second

    def update_core_matrix(self,s,a,s_):
        '''
        A function that performs step 12 and 13.
        '''
        #While the line below is in Algorithm 1, when using the Sherman Morrison update it is not needed.
        #self.A = self.A + np.outer(self.features_state_action[s,a],self.features_state_action[s,a])
        #Sherman Morrison Update (Rich's book Eqn 9.22)
        self.Ainv = self.Ainv - np.dot((np.outer(np.dot(self.Ainv,self.features_state_action[s,a]) \
                 ,self.features_state_action[s,a])),self.Ainv) / \
                    (1 + np.dot(np.dot(self.features_state_action[s,a],self.Ainv),self.features_state_action[s,a]))
        #The summation step of Eqn (2)
        self.sums = self.sums + np.outer(self.features_state_action[s,a],self.features_next_state[s_])
        #Here (K_psi)^-1 was pre-computed during the initialization. See Eqn (2)
        self.M = np.matmul(np.matmul(self.Ainv,self.sums),self.Kinv)

    def name(self):
        return 'Running: UC_MatrixRL'
    
    def getweightedL1(self):
        '''
        Returns the weighed L1 norm of the current estimated model and the true environment model
        '''
        self.weights = self.count
        self.true_p = np.zeros((self.env.nState,self.env.nAction,self.env.nState))
        for s in range(self.env.nState):
            for a in range(self.env.nAction):
                for s_ in range(self.env.nState):
                    self.true_p[s,a,s_] = self.env.P[s,a][s_]
                    
                    #for numerical stability
                    if sum(self.count[s,a,:]) == 0:
                        self.weights[s,a,s_] = self.count[s,a,s_]/1.0
                        
                    else:
                        self.weights[s,a,s_] = self.count[s,a,s_]/sum(self.count[s,a,:])
                        
        self.weights = self.weights.reshape(env.nState*env.nAction*env.nState)
        self.true_p = self.true_p.reshape(env.nState*env.nAction*env.nState)
        self.theta = self.M.reshape(env.nState*env.nAction*env.nState)
        temp = 0
        for i in range(env.nState*env.nAction*env.nState):
            temp += abs(self.theta[i]-self.true_p[i])*self.weights[i]
        return temp
            

    def run(self):
        '''
        Simulates the agent interacting with the environment over K episodes
        Input: Nothing
        Output: A Kx1 reward vector and a Kx1 model error vector
        '''
        R = 0
        Rvec = []
        #Stores count to be used in Weighted L1 Norm
        self.count = np.zeros((env.nState,env.nAction,env.nState))
        self.model_error = np.zeros(self.N)
        print(self.name())
        for n in tqdm(range(1,self.N+1)):
            self.env.reset()
            done = 0
            self.compute_Q(n)
            while not done:
                s = self.env.state
                h = self.env.timestep
                a = self.act(s,h)
                r,s_,done = self.env.advance(a)
                self.count[s,a,s_] += 1
                R += r
                self.update_core_matrix(s,a,s_)
            Rvec.append(R)
            self.model_error[n-1] = self.getweightedL1()
        return Rvec

In [7]:
class EGRL_VTR(object):
    '''
    Eps-Greedy VTR as described in the paper Model-Based RL with
    Value-Target Regression
    The algorithm assumes that the rewards are in the [0,1] interval.
    '''
    def __init__(self,env,K,eps):
        self.env = env
        self.K = K
        self.epsilon = eps
        # Here the dimension (self.d) for the Tabular setting is |S x A x S|
        self.d = self.env.nState * self.env.nAction * self.env.nState
        # In the tabular setting the basis models is just the dxd identity matrix
        self.P_basis = np.identity(self.d)
        self.Q = {(h,s,a): 0.0 for h in range(self.env.epLen) for s in self.env.states.keys() \
                   for a in range(self.env.nAction)}
        self.V = {(h,s): 0.0 for s in self.env.states.keys() for h in range(env.epLen + 1)} # self.V[env.epLen] stays zero
        self.sigma = {}
        self.state_idx = {}
        self.createIdx()
        self.lam = 1.0
        #Self.L is no longer need, but will keep for now.
        self.L = 1.0
        self.M = np.identity(self.d)*self.lam
        self.Minv = np.identity(self.d)*(1/self.lam)
        self.w = np.zeros(self.d)
        self.theta = np.dot(self.Minv,self.w)
        self.delta = 1/self.K
        self.m_2 = np.sqrt(self.env.nState*self.env.nAction)
        self.d1 = env.nState * env.nAction

    def feature_vector(self,s,a,h):
        '''
        Returning sum_{s'} V[h+1][s'] P_dot(s'|s,a),
        with V stored in self.
        Inputs:
            s - the state
            a - the action
            h - the current timestep within the episode
        '''
        sums = np.zeros(self.d)
        for s_ in self.env.states.keys():
            #print(s,s_)
            sums += self.V[h+1,s_] * self.P_basis[self.sigma[(s,a,s_)]]
        return sums

    def proj(self, x, lo, hi):
        '''Projects the value of x into the [lo,hi] interval'''
        return max(min(x,hi),lo)

    def update_Q(self,s,a,k,h):
        '''
        A function that updates both Q and V, Q is updated according to equation 4 and
        V is updated according to equation 2
        Inputs:
            s - the state
            a - the action
            k - the current episode
            h - the current timestep within the episode
        Currently, does not properly compute the Q-values but it does seem to learn theta_star
        '''
        #Here env.R[(s,a)][0] is the true reward from the environment
       
        X = self.feature_vector(s,a,h)
        self.Q[h,s,a] = self.proj(self.env.R[(s,a)][0] + np.dot(X,self.theta), 0, self.env.epLen)
        self.V[h,s] = max(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))

    def update_Qend(self,k):
        '''
        Updates the Value functions at the end of each episode.
        '''
        for h in range(self.env.epLen-1,-1,-1):
            for s in self.env.states.keys():
                for a in range(self.env.nAction):
                    #Here env.R[(s,a)][0] is the true reward from the environment
                    
                    self.update_Q(s,a,k,h)
                self.V[h,s] = (1 - self.epsilon/2) * max(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)])) \
                                + self.epsilon/2 * min(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))

    def update_stat(self,s,a,s_,h):
        '''
        A function that performs steps 9-13 of algorithm 1
        Inputs:
            s - the current state
            a - the action
            s_ - the next state
            k - the current episode
            h - the timestep within episode when s was visited (starting at zero)
        '''
        
        X = self.feature_vector(s,a,h)
    
        y = self.V[h+1,s_]
        
        self.M = self.M + np.outer(X,X)
        #Sherman-Morrison Update for M^{-1}, saves signficant computation
        self.Minv = self.Minv - np.dot((np.outer(np.dot(self.Minv,X),X)),self.Minv) / \
                    (1 + np.dot(np.dot(X,self.Minv),X))
        
        self.w = self.w + y*X

    def update_param(self):
        '''
        Updates our approximation of theta_star at the end of each episode
        '''
        #Step 15
        self.theta = np.matmul(self.Minv,self.w)

    def act(self,s,h,k):
        '''
        Returns the eps-greedy action with respect to Q_{h,k}(s,a) for a \in A
        Inputs:
            s - the current state
            h - the current timestep within the episode
        '''
        c = np.random.uniform()
        if c < self.epsilon:
            return np.random.choice(np.array([0,1]))
        else:
            return self.env.argmax(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))
        

    def createIdx(self):
        '''
        A simple function that creates sigma according to Appendix B.
        Here sigma is a dictionary who inputs is a tuple (s,a,s') and stores
        the interger index to be used in our basis model P.
        '''
        i = 0
        j = 0
        k = 0
        for s in self.env.states.keys():
            self.state_idx[s] = int(j)
            j += 1
            for a in range(self.env.nAction):
                for s_ in self.env.states.keys():
                    self.sigma[(s,a,s_)] = int(i)
                    i += 1
    
    def getweightedL1(self):
        '''
        Returns the weighed L1 norm of the current estimated model and the true environment model
        '''
        self.weights = self.count
        self.true_p = np.zeros((self.env.nState,self.env.nAction,self.env.nState))
        for s in range(self.env.nState):
            for a in range(self.env.nAction):
                for s_ in range(self.env.nState):
                    self.true_p[s,a,s_] = self.env.P[s,a][s_]
                    
                    #for numerical stability
                    if sum(self.count[s,a,:]) == 0:
                        self.weights[s,a,s_] = self.count[s,a,s_]/1.0
                        
                    else:
                        self.weights[s,a,s_] = self.count[s,a,s_]/sum(self.count[s,a,:])
                        
        self.weights = self.weights.reshape(env.nState*env.nAction*env.nState)
        self.true_p = self.true_p.reshape(env.nState*env.nAction*env.nState)
        temp = 0
        for i in range(env.nState*env.nAction*env.nState):
            temp += abs(self.theta[i]-self.true_p[i])*self.weights[i]
        return temp
            
        



    def run(self):
        '''
        Simulates the agent interacting with the environment over K episodes
        Input: Nothing
        Output: A Kx1 reward vector and a Kx1 model error vector
        '''
        R = []
        reward = 0.0
        #print(self.name())
        #Stores count to be used in Weighted L1 Norm
        self.count = np.zeros((env.nState,env.nAction,env.nState))
        self.model_error = np.zeros(self.K)
        for k in (range(1,self.K+1)):
            self.env.reset()
            done = 0
            while done != 1:
                s = self.env.state
                h = self.env.timestep
                a = self.act(s,h,k)
                r,s_,done = self.env.advance(a)
                self.count[s,a,s_] += 1
                reward += r
                self.update_stat(s,a,s_,h)
            self.update_param()
            self.update_Qend(k)
            R.append(reward)
            self.model_error[k-1] = self.getweightedL1()
        return R

    def name(self):
        '''
        A function that prints the name of the Algorithm
        This function is called in the run(self) function
        ''' 
        return 'Running: EGRL_VTR'



In [11]:
def save():
    '''
    A function that saves the data for the a given experiment after each run
    right now this saves the data under the DeepTree experimental data folder
    '''
    np.save("paper_vtr_data_deeptree/R_vtr",R_vtr)
    np.save("paper_vtr_data_deeptree/R_mat",R_mat)
    np.save("paper_vtr_data_deeptree/R_avg_eg_vtr",R_avg_vtr)
    np.save("paper_vtr_data_deeptree/R_avg_mat",R_avg_mat)
    np.save("paper_vtr_data_deeptree/R_eg_vtr",R_eg)
    np.save("paper_vtr_data_deeptree/R_mat_eg",R_mat_eg)

    np.save("paper_vtr_data_deeptree/model_vtr",model_vtr)
    np.save("paper_vtr_data_deeptree/model_mat",model_mat)
    np.save("paper_vtr_data_deeptree/model_eg_vtr",model_eg_vtr)
    np.save("paper_vtr_data_deeptree/model_eg_mat",model_eg_mat)
    
    np.save("paper_vtr_data_deeptree/sem_vtr",sem_vtr)
    np.save("paper_vtr_data_deeptree/sem_mat",sem_mat)


#Environment Episode
K = 10000
#Controls how many different states on would like to test on 
Trials = 3
#The number of Epsilon Greedy experiments to average over
Runs = 5
#The epsilon for the Eps Greedy methods
eps = 0.1

#Initialize Reward Vector for both UCRL-VTR and UC-MatrixRL
R_vtr = np.zeros((Trials,K))
R_mat = np.zeros((Trials,K))

#Initialize Reward Matrix for both EGRL-VTR and EG-Freq
R_eg = np.zeros((Trials,Runs,K))
R_mat_eg = np.zeros((Trials,Runs,K))

#A Matrix that stores the AVERAGE reward for both EGRL-VTR and EG-Freq
R_avg_vtr = np.zeros((Trials,K))
R_avg_mat = np.zeros((Trials,K))

#The SEM of the reward for the Epsilon Greedy Methods
sem_vtr = np.zeros((Trials,K))
sem_mat = np.zeros((Trials,K))

#Stores the Weighted L1 Error for all algorithms
model_vtr = np.zeros((Trials,K))
model_mat = np.zeros((Trials,K))
model_eg_vtr = np.zeros((Trials,K))
model_eg_mat = np.zeros((Trials,K))

#Loops through multiple different states of a given environment
for s in tqdm(range(Trials)):
    
    #Initialize the environment for testing
    env = make_MDP(epLen = 2,nBottom = (s+1)*4)
    #env = make_riverSwim(epLen=4*(s+3),nState = s+3)
    
    #Runs UCRL-VTR and stores Reward/Model Error
    agent_vtr = UCRL_VTR(env,K)
    R_vtr[s-3,:] = agent_vtr.run()
    model_vtr[s-3,:] = agent_vtr.model_error
    
    #Runs UC-MatrixRL and stores Reward/Model Error
    agent_mat = UC_MatrixRL(env,K)
    R_mat[s-3,:] = agent_mat.run()
    model_mat[s-3,:] = agent_mat.model_error
    
    #Initialize a vector that stores model error to be averaged for the Eps-Greedy Methods
    model_err_avg_vtr = np.zeros(K)
    model_err_avg_mat = np.zeros(K)
    
    #Loops through multiple runs of the Eps Greedy Algorithms
    for run in tqdm(range(Runs)):
        
        #Runs EGRL-VTR and stores Reward/Model Error
        agent_eg = EGRL_VTR(env,K,eps)
        R_eg[s,run,:] = agent_eg.run()
        model_err_avg_vtr += agent_eg.model_error
        
        #Runs EG-MatrixRL and stores Reward/Model Error
        agent_mat_eg = EG_MatrixRL(env,K,eps)
        R_mat_eg[s,run,:] = agent_mat_eg.run()
        model_err_avg_mat += agent_mat_eg.model_error
        
    #Averages the Model Error for the Eps-Greedy methods    
    model_eg_vtr[s,:] = model_err_avg_vtr/Runs
    model_eg_mat[s,:] = model_err_avg_mat/Runs
    
    #Averages the Reward for the Eps-Greedy methods     
    R_avg_vtr[s,:] = np.mean(R_eg[s,:,:],axis=0)
    R_avg_mat[s,:] = np.mean(R_mat_eg[s,:,:],axis=0)
    
    #Computes the SEM for the Eps-Greedy methods
    sem_vtr[s,:] = scipy.stats.sem(R_eg[s,:,:],axis=0)
    sem_mat[s,:] = scipy.stats.sem(R_mat_eg[s,:,:],axis=0)
    
    #Saves the data at the end of each "Trial"
    save()



Running: UCRL_VTR



Running: UC_MatrixRL



Running: UCRL_VTR



Running: UC_MatrixRL



Running: UCRL_VTR



Running: UC_MatrixRL
